In [3]:
import quantstats as qs
import pandas as pd
import numpy as np

date_index = pd.date_range(start='2020-01-01 00:00', periods=1000)
stock_returns = pd.Series((np.random.random(size=1000) - 0.5) / 10, index=date_index)

# 可视化并导出html文件
qs.reports.html(stock_returns, output="performance.html", download_filename="my_quant_stats.html")

# 指标计算
adj_result = qs.stats.adjusted_sortino(stock_returns)

# rolling 指标
stats_rolling_sharpe = qs.stats.rolling_sharpe(stock_returns, rolling_period=20)

# 其他指标
stats_monthly_returns = qs.stats.monthly_returns(stock_returns)

print("可计算的全部指标：", [f for f in dir(qs.stats) if f[0] != '_'])
print("可用于绘图的全部指标：", [f for f in dir(qs.stats) if f[0] != '_'])

from IPython.display import IFrame
IFrame(src='my_quant_stats.html', width=1000, height=600)

可计算的全部指标： ['adjusted_sortino', 'autocorr_penalty', 'avg_loss', 'avg_return', 'avg_win', 'best', 'cagr', 'calmar', 'common_sense_ratio', 'comp', 'compare', 'compsum', 'conditional_value_at_risk', 'consecutive_losses', 'consecutive_wins', 'cpc_index', 'cvar', 'distribution', 'drawdown_details', 'expected_return', 'expected_shortfall', 'exposure', 'gain_to_pain_ratio', 'geometric_mean', 'ghpr', 'greeks', 'implied_volatility', 'information_ratio', 'kelly_criterion', 'kurtosis', 'max_drawdown', 'monthly_returns', 'omega', 'outlier_loss_ratio', 'outlier_win_ratio', 'outliers', 'payoff_ratio', 'pct_rank', 'probabilistic_adjusted_sortino_ratio', 'probabilistic_ratio', 'probabilistic_sharpe_ratio', 'probabilistic_sortino_ratio', 'profit_factor', 'profit_ratio', 'r2', 'r_squared', 'rar', 'recovery_factor', 'remove_outliers', 'risk_of_ruin', 'risk_return_ratio', 'rolling_greeks', 'rolling_sharpe', 'rolling_sortino', 'rolling_volatility', 'ror', 'serenity_index', 'sharpe', 'skew', 'smart_sharpe', 

In [5]:
from dagster import asset, get_dagster_logger, op, DynamicOut, graph, job, DynamicOutput

from qbot.engine.datafeed import mongo_utils


@op(out=DynamicOut())
def load_bondlist():
    cols = ['code', 'bond_short_name', 'stk_code', 'list_date', 'delist_date']
    filters = {col: 1 for col in cols}
    filters['_id'] = 0
    items = mongo_utils.get_db()['cb_basic'].find({}, filters)
    items = list(items)
    items = ['1', '2', '3']
    for idx, item in enumerate(items):
        yield DynamicOutput(item, mapping_key=str(idx))


@op
def update_factor_chg(item):
    print('step...')
    return item


@op(name='update_close')
def update_factor_close(item):
    print(item)
    return item


@graph
def task_graph(item):
    update_factor_close(item)
    results = update_factor_chg(item)
    return results


@op
def merge_datas(results):
    print('merging...')


@job
def cb_task_job():
    bonds = load_bondlist()
    results = bonds.map(task_graph)
    print(results.collect())
    merge_datas(results.collect())


one_code_job = task_graph.to_job('task_graph_job')

if __name__ == "__main__":
    result = cb_task_job.execute_in_process()

ModuleNotFoundError: No module named 'quant_project'